In [2]:
!pip install gym
!pip install keras
!pip install keras-rl2

Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 696 kB 101 kB/s            
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Created wheel for gym: filename=gym-0.24.1-py3-none-any.whl size=793137 sha256=295e6be38ad4e0f00a01831123e594937d69541355262f801975e321dc8b71b2
  Stored in directory: /home/prathamesh/.cache/pip/wheels/c4/fb/19/388995b88cb551717a8dff40c889172cd12fadf994216a0a22
Successfully built gym
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 52 kB 314 kB/s             


In [9]:
!pip install pygame

Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 21.9 MB 387 kB/s            


In [3]:
import gym
import random

In [5]:
env = gym.make("CartPole-v1")
states = env.observation_space.shape[0]
actions = env.action_space.n

In [11]:
episodes = 10;
for i in range(1, episodes+1):
    state = env.reset()
    done = False
    score = 0
    
    while not done:
        env.render()
        action = random.choice([0,1])
        n_state, reward, done, info = env.step(action)
        score +=reward
    print(f"episode :{i}, score : {score}")

episode :1, score : 47.0
episode :2, score : 11.0
episode :3, score : 14.0
episode :4, score : 15.0
episode :5, score : 14.0
episode :6, score : 13.0
episode :7, score : 11.0
episode :8, score : 17.0
episode :9, score : 17.0
episode :10, score : 20.0


In [21]:
import numpy as np
import tensorflow
# from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.optimizers import Adam

In [22]:
def build_model(states, actions):
    model = tensorflow.keras.Sequential()
    model.add(Flatten(input_shape=(1, states)))
    model.add(Dense(24, activation='relu'))
    model.add(Dense(24, activation='relu'))
    model.add(Dense(actions, activation='linear'))
    return model

In [23]:
model =  build_model(states, actions)

In [24]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_1 (Flatten)         (None, 4)                 0         
                                                                 
 dense_3 (Dense)             (None, 24)                120       
                                                                 
 dense_4 (Dense)             (None, 24)                600       
                                                                 
 dense_5 (Dense)             (None, 2)                 50        
                                                                 
Total params: 770
Trainable params: 770
Non-trainable params: 0
_________________________________________________________________


In [25]:
from rl.agents import DQNAgent
from rl.policy import BoltzmannQPolicy
from rl.memory import SequentialMemory

In [30]:
def build_agent(model, actions):
    policy = BoltzmannQPolicy()
    memory = SequentialMemory(limit=5000, window_length=1)
    dqn = DQNAgent(model=model, memory=memory, policy= policy, 
                   nb_actions = actions, nb_steps_warmup=10, target_model_update=1e-2)
    return dqn

In [31]:
dqn = build_agent(model, actions)

In [32]:
dqn.compile(Adam(learning_rate=1e-3), metrics=['mae'])

In [33]:
dqn.fit(env, nb_steps=50000, visualize=False, verbose=1)

Training for 50000 steps ...
Interval 1 (0 steps performed)


/home/prathamesh/.local/lib/python3.10/site-packages/keras/engine/training_v1.py:2067: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates=self.state_updates,


    1/10000 [..............................] - ETA: 2:20:03 - reward: 1.0000

/home/prathamesh/.local/lib/python3.10/site-packages/rl/memory.py:37: UserWarning: Not enough entries to sample without replacement. Consider increasing your warm-up phase to avoid oversampling!
  warnings.warn('Not enough entries to sample without replacement. Consider increasing your warm-up phase to avoid oversampling!')
/home/prathamesh/.local/lib/python3.10/site-packages/rl/memory.py:38: DeprecationWarning: This function is deprecated. Please call randint(1, 10 + 1) instead
  batch_idxs = np.random.random_integers(low, high - 1, size=size)


   45/10000 [..............................] - ETA: 1:53 - reward: 1.0000

/home/prathamesh/.local/lib/python3.10/site-packages/rl/memory.py:38: DeprecationWarning: This function is deprecated. Please call randint(1, 11 + 1) instead
  batch_idxs = np.random.random_integers(low, high - 1, size=size)
/home/prathamesh/.local/lib/python3.10/site-packages/rl/memory.py:38: DeprecationWarning: This function is deprecated. Please call randint(1, 12 + 1) instead
  batch_idxs = np.random.random_integers(low, high - 1, size=size)
/home/prathamesh/.local/lib/python3.10/site-packages/rl/memory.py:38: DeprecationWarning: This function is deprecated. Please call randint(1, 13 + 1) instead
  batch_idxs = np.random.random_integers(low, high - 1, size=size)
/home/prathamesh/.local/lib/python3.10/site-packages/rl/memory.py:38: DeprecationWarning: This function is deprecated. Please call randint(1, 14 + 1) instead
  batch_idxs = np.random.random_integers(low, high - 1, size=size)
/home/prathamesh/.local/lib/python3.10/site-packages/rl/memory.py:38: DeprecationWarning: This funct

10000/10000 [==============================] - 53s 5ms/step - reward: 1.0000
102 episodes - episode_reward: 95.529 [8.000, 482.000] - loss: 1.111 - mae: 18.977 - mean_q: 38.332

Interval 2 (10000 steps performed)
10000/10000 [==============================] - 48s 5ms/step - reward: 1.0000
37 episodes - episode_reward: 277.135 [40.000, 465.000] - loss: 1.123 - mae: 37.700 - mean_q: 75.717

Interval 3 (20000 steps performed)
10000/10000 [==============================] - 43s 4ms/step - reward: 1.0000
23 episodes - episode_reward: 430.739 [77.000, 500.000] - loss: 4.184 - mae: 44.451 - mean_q: 88.974

Interval 4 (30000 steps performed)
10000/10000 [==============================] - 43s 4ms/step - reward: 1.0000
24 episodes - episode_reward: 403.875 [122.000, 500.000] - loss: 8.168 - mae: 51.381 - mean_q: 102.746

Interval 5 (40000 steps performed)
10000/10000 [==============================] - 43s 4ms/step - reward: 1.0000
done, took 228.744 seconds


In [34]:
scores = dqn.test(env, nb_episodes=100, visualize=False)
print(np.mean(scores.history['episode_reward']))

Testing for 100 episodes ...
Episode 1: reward: 500.000, steps: 500
Episode 2: reward: 500.000, steps: 500
Episode 3: reward: 500.000, steps: 500
Episode 4: reward: 500.000, steps: 500
Episode 5: reward: 500.000, steps: 500
Episode 6: reward: 500.000, steps: 500
Episode 7: reward: 500.000, steps: 500
Episode 8: reward: 500.000, steps: 500
Episode 9: reward: 500.000, steps: 500
Episode 10: reward: 500.000, steps: 500
Episode 11: reward: 500.000, steps: 500
Episode 12: reward: 500.000, steps: 500
Episode 13: reward: 500.000, steps: 500
Episode 14: reward: 500.000, steps: 500
Episode 15: reward: 500.000, steps: 500
Episode 16: reward: 500.000, steps: 500
Episode 17: reward: 500.000, steps: 500
Episode 18: reward: 500.000, steps: 500
Episode 19: reward: 500.000, steps: 500
Episode 20: reward: 500.000, steps: 500
Episode 21: reward: 500.000, steps: 500
Episode 22: reward: 500.000, steps: 500
Episode 23: reward: 500.000, steps: 500
Episode 24: reward: 500.000, steps: 500
Episode 25: reward: 

In [35]:
_ = dqn.test(env, nb_episodes=10, visualize=True)

Testing for 10 episodes ...
Episode 1: reward: 500.000, steps: 500
Episode 2: reward: 500.000, steps: 500
Episode 3: reward: 500.000, steps: 500
Episode 4: reward: 500.000, steps: 500
Episode 5: reward: 500.000, steps: 500
Episode 6: reward: 500.000, steps: 500
Episode 7: reward: 500.000, steps: 500
Episode 8: reward: 500.000, steps: 500
Episode 9: reward: 500.000, steps: 500
Episode 10: reward: 500.000, steps: 500


In [36]:
dqn.save_weights('CartPole-v1_weight.h5f', overwrite=True)